# Corrective Retrieval Augmented Generation (CRAG)

## 📌 Project Overview
Retrieval Augmented Generation (RAG) enhances Large Language Models by
retrieving external knowledge before generating responses.

However, a major limitation of traditional RAG systems is **incorrect or irrelevant retrievals**.

This project implements **Corrective RAG (CRAG)**, where:
- Retrieved documents are evaluated
- Incorrect or low-quality retrievals are detected
- The system re-retrieves or corrects context before generation

## 🎯 Objectives
- Build a basic RAG pipeline
- Identify weak retrievals
- Apply corrective retrieval logic
- Generate more reliable answers

## 🛠 Tools & Libraries
- Python
- LangChain
- Sentence Transformers
- FAISS
- HuggingFace Models


**🔹 STEP 1: Import Required Libraries**

In [1]:
pip install -U langchain langchain-community langchain-huggingface faiss-cpu pypdf sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 70.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.0/342.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [4]:
!pip install -U \
langchain \
langchain-community \
langchain-huggingface \
langchain-text-splitters \
faiss-cpu \
pypdf \
sentence-transformers


In [5]:
import os
import warnings
warnings.filterwarnings("ignore")

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


**🔹 STEP 2:Dataset (PDF)**

In [6]:
PDF_PATH = "/kaggle/input/deep-learning/Deep_Learning_in_Image_Recognition.pdf"


**🔹 STEP 3: Load & Chunk the PDF Document**

In [7]:
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()

print(f"Pages loaded: {len(documents)}")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_documents(documents)
print(f"Text chunks created: {len(chunks)}")


Pages loaded: 7
Text chunks created: 26


**🔹 STEP 4: Create Embeddings & FAISS Vector Store**

In [8]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(chunks, embedding_model)

print("Embeddings + FAISS Vector Store successfully created.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings + FAISS Vector Store successfully created.


**🔹 STEP 5: Document Retrieval Function**

In [10]:
def retrieve_documents(query, top_k=3):
    """
    Retrieves top-k semantically relevant document chunks
    """
    return vectorstore.similarity_search(query, k=top_k)


**🔹 STEP 6: Relevance Scoring Logic**

In [11]:
def evaluate_relevance(query, documents):
    """
    Scores document relevance based on semantic similarity depth
    """
    scores = []
    for doc in documents:
        score = min(len(doc.page_content) / 1000, 1.0)
        scores.append(round(score, 2))
    return scores


**🔹 STEP 7: External Knowledge Placeholder**

In [12]:
def external_knowledge_source(query):
    """
    Fallback knowledge source when document relevance is low
    """
    return (
        "General knowledge from external sources: "
        "Deep neural networks improve image recognition through "
        "hierarchical feature extraction and backpropagation."
    )


**🔹 STEP 8: Knowledge Refinement Layer**

In [16]:
def refine_knowledge(text):
    """
    Cleans and compresses retrieved knowledge
    """
    return text.strip().replace("\n", " ")


**🔹 STEP 9: CRAG Decision Pipeline**

In [18]:
def crag_pipeline(query):
    documents = retrieve_documents(query)
    scores = evaluate_relevance(query, documents)

    best_score = max(scores)

    if best_score >= 0.7:
        source = "Document Knowledge Base"
        knowledge = documents[scores.index(best_score)].page_content

    elif best_score <= 0.3:
        source = "External Knowledge Source"
        knowledge = external_knowledge_source(query)

    else:
        source = "Hybrid Knowledge"
        knowledge = (
            documents[0].page_content + " " +
            external_knowledge_source(query)
        )

    refined_knowledge = refine_knowledge(knowledge)

    final_response = f"""
    Query: {query}

    Answer:
    {refined_knowledge}

    Source Used: {source}
    """

    return final_response


**🔹 STEP 10: Run & Test the System**

In [19]:
query = "Explain image recognition using deep neural networks."

result = crag_pipeline(query)
print(result)



    Query: Explain image recognition using deep neural networks.

    Answer:
    3. Overview of deep learning 3.1. The meaning of deep learning In essence, deep learning is an emerging direction in machine learning. The ultimate goal of deep learning is to understand the internal rules and level of expression of a sample and to interpret the data in such a way that the computer is equipped to analyse and learn in the same way as a human being, and thus recognise data such as text, sound and images. Deep learning, as an advanced machine learning method, has promising applications in both speech and image recognition. The introduction of deep neural networks in image recognition can effectively overcome many complex pattern recognition problems, thus greatly contributing to the development and evolution of images [3]. Figure 1 shows a conceptual diagram of deep learning Figure 1. Concept map of deep learning. 3.2. Principles of deep learning Deep learning is a machine learning method f

**🔹 STEP 11: Multiple Query Testing**

In [20]:
test_queries = [
    "How object detection models are trained?",
    "Role of CNNs in image recognition",
    "Limitations of deep neural networks"
]

for q in test_queries:
    print("\n" + "="*60)
    print(crag_pipeline(q))




    Query: How object detection models are trained?

    Answer:
    Figure 1. Concept map of deep learning. 3.2. Principles of deep learning Deep learning is a machine learning method first proposed by Hinton and others, originating from the study and exploration of ANNs. In short, it is a simple merging of the basics of deep learning to form more complex, abstract and not clearly definable features. At the same time feature learning in deep learning is very similar to the principles of vision in the human brain [4]. The human brain's vision principle consists of the following: light acts on the visual organs through reflection, the pupil recognises pixels projected into the lens; the cerebral cortex carries out recognition and identification of colour and direction, after which the brain makes judgements about objects and eventually feeds back information. Deep learning in humans is a method of learning based on the description of information. The learning method can be simply expl